In [ ]:
from ultralytics import YOLO
import cv2
import os
import numpy as np

# 📌 YOLO 모델 로드
model = YOLO("yolov8m.pt", task="detect")

# 📌 감지할 이미지 폴더
input_folder = "C:/Users/wlsth/OneDrive/Desktop/MS_Project/data/mixed_data_cracked_only"
output_folder = "C:/Users/wlsth/OneDrive/Desktop/MS_Project/data/detected_part_bounding_box"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 📌 Canny Edge 적용 + 파손 면적 계산 함수
def apply_canny(roi, x_offset, y_offset, original_image):
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)  # 흑백 변환
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)  # 노이즈 제거
    edges = cv2.Canny(blurred, 50, 150)  # Canny Edge Detection 적용

    # 파손된 픽셀 수 계산
    damage_pixels = np.count_nonzero(edges)
    total_pixels = edges.size
    damage_ratio = (damage_pixels / total_pixels) * 100

    # 원본 이미지에 Canny 결과 적용 (ROI 내부만)
    original_image[y_offset:y_offset + roi.shape[0], x_offset:x_offset + roi.shape[1]] = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    print(f"📌 파손 면적 비율: {damage_ratio:.2f}%")
    return edges, damage_ratio

# 📌 YOLO Object Detection 실행 함수
def detect_damage(image_path):
    image = cv2.imread(image_path)
    results = model(image, conf=0.5, iou=0.5)  # YOLO 감지 수행

    for result in results:
        for box in result.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # 바운딩 박스 그림
            
            # ROI (바운딩 박스 내부) Canny 적용
            roi = image[y1:y2, x1:x2]
            apply_canny(roi, x1, y1, image)

    output_path = os.path.join(output_folder, os.path.basename(image_path))
    cv2.imwrite(output_path, image)
    print(f"✅ Object Detection + Canny 완료: {output_path}")

# 📌 YOLO + OpenCV Canny 실행
for filename in os.listdir(input_folder):
    if filename.endswith((".jpg", ".png", ".jpeg")):
        image_path = os.path.join(input_folder, filename)
        detect_damage(image_path)